In [52]:
import os
import sys
import re
import math

import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk.corpus import stopwords

import fasttext
import pkg_resources
import kenlm

In [ ]:
gloveFile = "data\\glove.6B.50d.txt"

### Transformer Paper



yelp_acc_path = 'acc_yelp.bin'
yelp_ppl_path = 'ppl_yelp.binary'
yelp_ref0_path = 'yelp.refs.0'
yelp_ref1_path = 'yelp.refs.1'


yelp_acc_file = pkg_resources.resource_stream(resource_package, yelp_acc_path)
yelp_ppl_file = pkg_resources.resource_stream(resource_package, yelp_ppl_path)
yelp_ref0_file = pkg_resources.resource_stream(resource_package, yelp_ref0_path)
yelp_ref1_file = pkg_resources.resource_stream(resource_package, yelp_ref1_path)


self.yelp_ref = []
with open(yelp_ref0_file.name, 'r') as fin:
    self.yelp_ref.append(fin.readlines())
with open(yelp_ref1_file.name, 'r') as fin:
    self.yelp_ref.append(fin.readlines())
self.classifier_yelp = fasttext.load_model(yelp_acc_file.name)
self.yelp_ppl_model = kenlm.Model(yelp_ppl_file.name)

In [ ]:
# Style check from fastText model

def yelp_style_check(self, text_transfered, style_origin):
        text_transfered = ' '.join(word_tokenize(text_transfered.lower().strip()))
        if text_transfered == '':
            return False
        label = self.classifier_yelp.predict([text_transfered])
        style_transfered = label[0][0] == '__label__positive'
        return (style_transfered != style_origin)
    
# Checking the accuracy for different styles
    
def yelp_acc_b(self, texts, styles_origin):
        assert len(texts) == len(styles_origin), 'Size of inputs does not match!'
        count = 0
        for text, style in zip(texts, styles_origin):
            if self.yelp_style_check(text, style):
                count += 1
        return count / len(texts)

def yelp_acc_0(self, texts):
        styles_origin = [0] * len(texts)
        return self.yelp_acc_b(texts, styles_origin)

def yelp_acc_1(self, texts):
        styles_origin = [1] * len(texts)
        return self.yelp_acc_b(texts, styles_origin)

In [40]:
# Initialize the NLTK model
def nltk_bleu(texts_origin, text_transfered):
        texts_origin = [word_tokenize(text_origin.lower().strip()) for text_origin in texts_origin]
        text_transfered = word_tokenize(text_transfered.lower().strip())
        return sentence_bleu(texts_origin, text_transfered) * 100

# Check the BLEU diff between original & transferred text
def self_bleu_b(self, texts_origin, texts_transfered):
        assert len(texts_origin) == len(texts_transfered), 'Size of inputs does not match!'
        sum = 0
        n = len(texts_origin)
        for x, y in zip(texts_origin, texts_transfered):
            sum += self.nltk_bleu([x], y)
        return sum / n

In [ ]:
# Measures perplexity of language model

def yelp_ppl(self, texts_transfered):
        texts_transfered = [' '.join(word_tokenize(itm.lower().strip())) for itm in texts_transfered]
        sum = 0
        words = []
        length = 0
        for i, line in enumerate(texts_transfered):
            words += [word for word in line.split()]
            length += len(line.split())
            score = self.yelp_ppl_model.score(line)
            sum += score
        return math.pow(10, -sum / length)

### ARAE

In [ ]:

def train_ngram_lm(kenlm_path, data_path, output_path, N):
    """
    Trains a modified Kneser-Ney n-gram KenLM from a text file.
    Creates a .arpa file to store n-grams.
    """
    # create .arpa file of n-grams
    curdir = os.path.abspath(os.path.curdir)
    #
    command = "bin/lmplz -o "+str(N)+" <"+os.path.join(curdir, data_path) + \
              " >"+os.path.join(curdir, output_path)
    os.system("cd "+os.path.join(kenlm_path, 'build')+" && "+command)

    load_kenlm()
    # create language model
    assert(output_path)  # captured by try..except block outside
    model = kenlm.Model(output_path)

    return model


def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        words = sent.strip().split()
        nll = np.sum([- math.log(math.pow(10.0, score)) for score, _, _ in lm.full_scores(sent, bos=True, eos=False)])
        word_count = len(words)
        total_wc += word_count
        total_nll += nll
    ppl = np.exp(total_nll / total_wc)
    return ppl


### Other

In [23]:
def cosine_distance_countvectorizer_method(s1, s2):
    
    # sentences to list
    allsentences = [s1 , s2]
    
    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance
    
    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    
    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    print('Similarity of two sentences are equal to ',round((1-cosine)*100,2),'%')
    return cosine

In [ ]:


def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    with open(gloveFile, encoding="utf8" ) as f:
        content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model


def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words

def cosine_distance_between_two_words(word1, word2):
    import scipy
    return (1- scipy.spatial.distance.cosine(model[word1], model[word2]))

def calculate_heat_matrix_for_two_sentences(s1,s2):
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    result_list = [[cosine_distance_between_two_words(word1, word2) for word2 in s2] for word1 in s1]
    result_df = pd.DataFrame(result_list)
    result_df.columns = s2
    result_df.index = s1
    return result_df

def cosine_distance_wordembedding_method(s1, s2):
    import scipy
    vector_1 = np.mean([model[word] for word in preprocess(s1)],axis=0)
    vector_2 = np.mean([model[word] for word in preprocess(s2)],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    print('Word Embedding method with a cosine distance asses that our two sentences are similar to',round((1-cosine)*100,2),'%')

def heat_map_matrix_between_two_sentences(s1,s2):
    df = calculate_heat_matrix_for_two_sentences(s1,s2)
    import seaborn as sns
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(5,5)) 
    ax_blue = sns.heatmap(df, cmap="YlGnBu")
    # ax_red = sns.heatmap(df)
    print(cosine_distance_wordembedding_method(s1, s2))
    return ax_blue

## Code

In [27]:
path = "/Users/spencerbraun/Documents/Stanford/CS 230 - Deep Learning/Project/style-transformer/outputs/soph/model_iteration_lr_0.0001/"
with open(path + "gold_text.txt") as f:
    gold = f.readlines()
    
with open(path + "rev_output_0.txt") as f:
    rev0 = f.readlines()
    
with open(path + "raw_output_0.txt") as f:
    raw0 = f.readlines()

In [33]:
def process(sent):
    sent = sent.strip().replace('<pad>', '').strip()
    return sent

In [36]:
gold = list(map(process, gold))
rev0 = list(map(process, rev0))
raw0 = list(map(process, raw0))

In [46]:
def bleu_sent(originText, transferredText):
        texts_origin = [
            word_tokenize(text.lower().strip()) 
            for text in originText
        ]
        text_transfered = word_tokenize(transferredText.lower().strip())
        return sentence_bleu(texts_origin, text_transfered) * 100
    


def bleu_avg(originText, transferredText):
        assert len(originText) == len(transferredText), 'Size of inputs does not match!'
        sum = 0
        n = len(originText)
        for x, y in zip(originText, transferredText):
            sum += bleu_sent([x], y)
        return sum / n

In [49]:
self_bleu_b(gold, raw0)

83.02805322367601

In [55]:
def load_kenlm():
    global kenlm
    import kenlm

def train_ngram_lm(kenlm_path, data_path, output_path, N):
    """
    FROM ARAE
    Trains a modified Kneser-Ney n-gram KenLM from a text file.
    Creates a .arpa file to store n-grams.
    """
    # create .arpa file of n-grams
    curdir = os.path.abspath(os.path.curdir)
    #
    command = "bin/lmplz -o "+str(N)+" <"+os.path.join(curdir, data_path) + \
              " >"+os.path.join(curdir, output_path)
    os.system("cd "+os.path.join(kenlm_path, 'build')+" && "+command)

    load_kenlm()
    # create language model
    assert(output_path)  # captured by try..except block outside
    model = kenlm.Model(output_path)

    return model

In [56]:
train_ngram_lm('kenlm', 'data/processed/soph_train_small.txt', 'kenlm_output', 5)

OSError: Cannot read model 'kenlm_output' (util/file.cc:183 in std::size_t util::PartialRead(int, void *, std::size_t) threw FDException because `ret < 0'. Is a directory in fd 97 while reading 6 bytes in file /Users/spencerbraun/Documents/Stanford/CS 230 - Deep Learning/Project/CS_230_Project/kenlm_output)

In [ ]:
def get_ppl(lm, sentences):
    """
    Assume sentences is a list of strings (space delimited sentences)
    """
    total_nll = 0
    total_wc = 0
    for sent in sentences:
        words = sent.strip().split()
        nll = np.sum([- math.log(math.pow(10.0, score)) for score, _, _ in lm.full_scores(sent, bos=True, eos=False)])
        word_count = len(words)
        total_wc += word_count
        total_nll += nll
    ppl = np.exp(total_nll / total_wc)
    return ppl